In [59]:
# OCR - Tesseract

from PIL import Image
import pytesseract
import cv2
import re
import os

In [71]:
# https://blog.naver.com/PostView.nhn?blogId=tommybee&logNo=221307497468&parentCategoryNo=&categoryNo=54&viewDate=&isShowPopularPosts=true&from=search
# Define config parameters.
# '-l eng'  for using the English language
# '--oem 1' for using LSTM OCR Engine
# config = ('-l kor+eng --oem 1 --psm 3')
# config = ('-l kor --oem 2 --psm 3 ') # ERROR, oem : lagacy + lstm, no support kor
# config = ('-l kor --oem 1 --psm 1') # oem3 와 비슷

# kor > kor+eng, 영어를 함께 인식하면 오류율이 높아진다
# psm 정확도  6 >>> 12,11 > 3
config = ('-l kor --oem 1 --psm 6') 
image_folder = '/images/video_receipt'

def ocr_tesseract():
    try:
        all_file = get_all_file(image_folder)
        result = []
        
        for image_file in all_file:
#             image_file = 'images/reciept1.jpg'
            img = cv2.imread(image_file)
            img = img_preprocessing(img)
            text = pytesseract.image_to_string(img, config=config)
            text = text_processing(text)
            
            result.append(text)
        
        # 이미지 표시
#         img_show(img)
        
        # 결과 출력
#         print(text)
        print(result)
        print(result[0])

    except Exception as ex:
        print('ERROR:', ex)

# 해당 폴더에 있는 모든 파일들의 절대 경로+이름을 리턴한다        
def get_all_file(image_folder):
    path_files = os.getcwd() 
    file_name = next(os.walk('.'+image_folder))[2]
    all_file = [path_files + image_folder + '/' + file for file in file_name]
    
    return all_file

# 중복된 공백과 줄바꿈을 없앤다        
def text_processing(text):       
#     text = " ".join(text.split()) # 리스트를 문자열로 변환, 줄바꿈까지 다 없애버린다, 한줄씩의 줄바꿈은 필요
    text = re.sub(' +', ' ', text) # 중복 공백 제거
    text = text.replace('\n\n', '\n') # 중복 줄바꿈 제거
    text = text.split('\n') # \n 단위로 리스트로 변경
    
    return text

def img_show(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
        
#1. 아래의 전처리는 정확도를 떨어뜨린다, 가격이 잘 뽑히지 않음     
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
def img_preprocessing(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    #그레이 스케일 변환
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]  #배경에서 전경 텍스트를 분할하기 위해 임계값을 사용
    
    return gray
    
if __name__ == '__main__':
    ocr_tesseract()

[['0 시세', '어등 기막보 007144440084', '써000000000000', ' -회국치브이느건세버- 이시이선인 1 211 107/ "', '아어 히 스불머즐 1014210400)', '튀', '벅재 564:71-00289 191: 064002008', '제주택별자차도 재수사 모로4길 ?니', '디01 2811908076 60690 009 00000', '기총 -103처1 -7 91"', '시! 1!쏘그 9090 44 아이 4184', '비@시- 거014140711 }규거0뻐 (0시베)', '까오 혀 이 17.000원', '4440@11 본기시 보낸', '인 기 17. 0000띠', '흉배 30002 ㅣ', '매빌사면: 때콕민', '기뱅성번호: 00094008410', '제크31드', '강사합니다'], ['000 기에', '2밸살빅 키방섬송가 4제봐나8깅무', '!으드후들 바바 버 1', ' 윈국티기트게마 이니어리아 시바 으시이!', ':', '모책을 -스원다@ 10400)', '깨텐번수신', '백재현 564-71-00201 1011 464/020928', '제쑤톡벌자저도 제주시 통분루44 이', '110: 201100800 70560 0008 9 7', '))! ## <6323411 7211', '시! 쑤이 490 4040 1 4189', '0041 1티테4080 10.42.00 (회시복|', '나 서 0 아 17.000쁜', '140400@71 누기세 개', '인 4 17, 0009시', '홀빈06: 연넌터던더 ㅣ', '매밀사핑: 배곡만', '가뱅침버로 00004006.0보', '00', '자사첩니다'], ['배설벅 키벌심수조기 싱세외 디뮤징우', '도룩를 비00100서바러', '1: 12미내0심1 0\\지 1 9비심', ': 위국휘용차느겸세주 이브 그바 바 1"', '1고 랙 월! 스 01 1042040)', '깨텐불수신', '백재원 58471-00201 191: 0042020070', '제주특벌사치도 제주시 풍문우44 7디', '110: 2